In [7]:
from glob import glob
import librosa
import numpy as np

from multiprocessing import Pool


# bird_classifier/bird and bird_classifier/background
background = glob("/mnt/md0/urbansounddb/UrbanSound8K/audio/**/*wav")
birds = glob('/mnt/md0/birdsong-recognition/train_audio/**/*mp3')




In [39]:
# define a function for the threaded work
def doIt(f):
    x_train=[]
    y_train =[]
    if("wav" in f):
        theclass=0
    else:
        theclass=1
    try:
        mp3, sr = librosa.load(f, sr = 22050//2)
        for i in range(0, mp3.shape[0], sr*3):
            x = mp3[i:i+sr*3]
            if(x.shape[0]== sr*3):
                x_train.append(x)
                y_train.append(theclass)
    except Exception as e:
        print(e)
    return x_train, y_train

if __name__ == '__main__':   
    p = Pool(30)    # create 30 computers 
    pooled_birds = p.map(doIt, birds[::50])  # send the function "doit" plus all the filenames to be processed to all the new threads
    pooled_background  = p.map(doIt, background)  # different offset (1) and a stride that is multiple of training.   Guaranteed to be unique

p.terminate()  # clean up otherwise it seems to cause problems with the kernel 

/home/gary/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/gary/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/gary/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/gary/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/gary/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.


In [41]:
print(len(pooled_birds), len(pooled_background))

428 8732


In [42]:
def unpoolData(pooled):
    xx= []  # temporary placeholdsrs
    yy= []  # to unpack results
    for f in pooled:
        for ff in f[0]:
            xx.append(ff)
        for ff in f[1]:
            yy.append(ff)

    xx=np.array(xx)
    yy = np.array(yy)
    return xx, yy
 

In [43]:
x_train, y_train = unpoolData(pooled_birds + pooled_background)

In [44]:
len(x_train), len(y_train)
x_train.shape

(16342, 33075)

In [45]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam, Adagrad

model=Sequential()
sr = 33075
model.add(Conv1D(16,3, input_shape=(sr,1), activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(32,3, padding='same', activation='relu'))
model.add(MaxPooling1D(4))
model.add(Conv1D(32,3, padding='same', activation='relu'))
model.add(MaxPooling1D(4))
model.add(Conv1D(64,3, activation='relu'))
model.add(MaxPooling1D(4))
model.add(Conv1D(128,3, activation='relu'))
model.add(Dropout(.1))
model.add(MaxPooling1D(3))
model.add(Conv1D(256,3, activation='relu'))
model.add(Dropout(.1))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(300, activation='relu'))
model.add(Dropout(.1))  
model.add(Dense(1, activation='sigmoid')) # multiclass classifier vs binary classifier would use sigmoid


# because softmax we use categorical
model.compile(loss='binary_crossentropy', optimizer=Adagrad(), metrics=['acc'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 33073, 16)         64        
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 16536, 16)         0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 16536, 32)         1568      
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 4134, 32)          0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 4134, 32)          3104      
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 1033, 32)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 1031, 64)         

In [50]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
model.fit(x_train, y_train, validation_split=.1, epochs=10)

Train on 14707 samples, validate on 1635 samples
Epoch 1/10
14707/14707 [==============================] - 10s 679us/step - loss: 0.1258 - acc: 0.9530 - val_loss: 0.9938 - val_acc: 0.8281
Epoch 2/10
14707/14707 [==============================] - 10s 678us/step - loss: 0.1189 - acc: 0.9566 - val_loss: 0.7940 - val_acc: 0.8385
Epoch 3/10
14707/14707 [==============================] - 10s 702us/step - loss: 0.1130 - acc: 0.9589 - val_loss: 0.6318 - val_acc: 0.8911
Epoch 4/10
14707/14707 [==============================] - 10s 682us/step - loss: 0.1046 - acc: 0.9629 - val_loss: 0.7606 - val_acc: 0.8673
Epoch 5/10
14707/14707 [==============================] - 10s 683us/step - loss: 0.0982 - acc: 0.9640 - val_loss: 0.9093 - val_acc: 0.8569
Epoch 6/10
14707/14707 [==============================] - 10s 673us/step - loss: 0.0927 - acc: 0.9666 - val_loss: 1.1247 - val_acc: 0.8300
Epoch 7/10
14707/14707 [==============================] - 10s 682us/step - loss: 0.0891 - acc: 0.9678 - val_loss: 1.1

In [47]:
np.sum(y_train)/y_train.shape[0]

0.5430791824746053